In [ ]:
# All unaccounted for variables in this file should be drawn from here (will put this in gitignore)
from wealth_modelling_variables import *

# Classes to help track overall wealth
from bank_account import *
from mortgage import *
from student_loan import *

# Functions to help model cashflow
from cost_modelling import *
from salary_modelling import *
from tax_modelling import *

# Oh and of course
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Initialising our 'account' and 'loan' objects
CurrentAccount = BankAccount(curent_account_balance, current_account_interest)
Mortgage = Mortgage(CurrentAccount, mortgage_deposit, initial_mortgage_value, repayment_term, monthly_payments, fixed_rate, fixed_rate_period)
StudentLoan = StudentLoan(CurrentAccount, initial_student_loan_value, student_loan_bands, student_loan_rates)
Pension = BankAccount(current_pension_value, pension_interest_rate)

In [ ]:
wealth_increments = []
current_account_vals = []
pension_vals = []
property_vals = []
mortgage_vals = []
student_loan_vals = []


costs = model_costs(years, monthly_rent)

salaries = model_salary(years, starting_salary, annual_salary_increase, bonus_percentage)
for i,salary in enumerate(salaries):
    # First, pay my salary into my current account
    CurrentAccount.pay_in(salary)
    
    # Consider the effect on our student loan
    student_loan_repayment = StudentLoan.make_minimum_repayment(salary)
    # A curious instance of loan whose interest rate is contingent on my salary...
    StudentLoan.compound_interest(salary)

    # Next, address our pension contribution
    Pension.pay_in(salary*(pension_contribution + employer_pension_contribution))
    Pension.compound_interest()
    # Now reduce the amount of income that is put forward for tax consideration
    salary *= (1 - pension_contribution)

    # Now apply income tax and NI
    tax = get_tax(salary, ni_bands, ni_rates, it_bands, it_rates)
    # Obviously this never actually reached our account but for this purpose its fine it was there briefly
    CurrentAccount.withdraw(tax)

    # Take out our mortgage payment
    mortgage_repayment = Mortgage.make_yearly_repayment()
    Mortgage.compound_interest()
    # Acknowledge that the underlying asset is growing in value!
    Mortgage.AssetValue *= 1.01

    # Get income from tenants in property
    tenant_income = tenant_monthly_rent * 12 * tenant_no
    # Strictly speaking a small amount of this is taxable, however lodgers are a thing
    CurrentAccount.pay_in(tenant_income)
    
    # Withdraw costs
    CurrentAccount.withdraw(costs[i])

    # Finally, compound interest on whatevers left
    CurrentAccount.compound_interest()

    current_account_vals += [CurrentAccount.value]
    pension_vals += [Pension.value]
    property_vals += [Mortgage.AssetValue]
    mortgage_vals += [Mortgage.loan_value]
    student_loan_vals += [StudentLoan.loan_value]
    wealth_increments += [CurrentAccount.value + Pension.value + Mortgage.AssetValue - Mortgage.loan_value - StudentLoan.loan_value]

In [ ]:
plt.figure(figsize=(25,10))
plt.tight_layout()

# Stacked area chart for assets
ax1 = plt.subplot(1,2,1)
plt.stackplot(
    np.arange(0,years,1)+2022,
    property_vals,
    pension_vals,
    current_account_vals,
    labels=[
        'Property Value',
        'Pension',
        'Current Account'
    ]
)
plt.plot(
    np.arange(0,years,1)+2022,
    wealth_increments,
    color='black',
    label='Net Wealth'
)
plt.xlabel('Years from FY 2022/23',fontsize=20)
plt.ylabel('Total Assets/Liabilities',fontsize=20)
plt.tick_params(labelsize=15)
l = plt.legend(loc=0,prop={'size': 15},title='Wealth Trajectories') 
plt.setp(l.get_title(),fontsize=15)

# Stacked area chart for liabilities
ax2 = plt.subplot(1,2,2,sharey=ax1)
plt.stackplot(
    np.arange(0,years,1)+2022,
    mortgage_vals,
    student_loan_vals,
    labels=[
        'Mortgage',
        'Student Loan'
    ]
)
plt.plot(
    np.arange(0,years,1)+2022,
    wealth_increments,
    color='black',
    label='Net Wealth'
)
plt.xlabel('Years from FY 2022/23',fontsize=20)
#plt.ylabel('Total Liabilities',fontsize=20)
plt.tick_params(labelsize=15, labelleft=False)
l = plt.legend(loc=0,prop={'size': 15},title='Debt Trajectories') 
plt.setp(l.get_title(),fontsize=15)

plt.show()